## Keras ile Sinir Ağlarına En Basit Giriş

> Bu not defteri, [Yeni Başlayanlar için YZ Müfredatı](http://github.com/microsoft/ai-for-beginners)'nın bir parçasıdır. Eksiksiz öğrenme materyalleri kümesi için kod deposunu ziyaret edin.

### Sinir Çerçeveleri

Sinir ağlarını eğitmek için çeşitli çerçeveler vardır. Ancak, hızlı bir başlangıç yapmak ve işlerin içeride nasıl çalıştığına dair fazla ayrıntıya girmek istemiyorsanız [Keras](https://keras.io/) kullanmayı düşünmelisiniz. Bu kısa eğitim başlamanıza yardımcı olacak ve işlerin nasıl yürüdüğünü daha iyi anlamak istiyorsanız - [Tensorflow ve Keras'a Giriş](IntroKerasTF.tr.ipynb) not defterine bakın.

### İşleri hazırlamak

Keras, Tensorflow 2.x çerçevesinin bir parçasıdır. Tensorflow'un 2.x.x sürümünün kurulu olduğundan emin olalım:
```
pip install tensorflow
```
veya
```
conda install tensorflow
```

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
print(f'Tensorflow sürümü = {tf.__version__}')
print(f'Keras sürümü" = {keras.__version__}')

## Temel Kavramlar: Tensör

**Tensör** çok boyutlu bir dizidir. Farklı veri türlerini temsil etmek için tensör kullanmak çok uygundur:
* 400x400 - siyah beyaz resim
* 400x400x3 - renkli resim 
* 16x400x400x3 - 16 adet renkli resimden minigrup
* 25x400x400x3 - 25 fps'lik videonun bir saniyesi
* 8x25x400x400x3 - 8 adet 1 saniyelik videodan minigrup

Tensörler, sinir ağı içindeki ağırlıkların yanı sıra, girdi/çıktı verilerini temsil etmek için de bize uygun bir yol sağlar.

## Örnek Problem

İkili sınıflandırma problemini ele alalım. Böyle bir soruna iyi bir örnek, boyutuna ve yaşına göre kötü ve iyi huylular arasında tümör sınıflandırması olabilir. Bazı örnek veriler oluşturarak başlayalım:

In [ ]:
np.random.seed(0) # tekrarlanabilirlik için tohumu seçin - rastgele varyasyonların etkilerini keşfetmek için değiştirin

n = 100
X, Y = make_classification(n_samples = n, n_features=2,
                           n_redundant=0, n_informative=2, flip_y=0.05,class_sep=1.5)
X = X.astype(np.float32)
Y = Y.astype(np.int32)

split = [ 70*n//100 ]
train_x, test_x = np.split(X, split)
train_labels, test_labels = np.split(Y, split)

In [ ]:
def plot_dataset(features, labels, W=None, b=None):
    # çizimi hazırlamak
    fig, ax = plt.subplots(1, 1)
    ax.set_xlabel('$x_i[0]$ -- (öznitelik 1)')
    ax.set_ylabel('$x_i[1]$ -- (öznitelik 2)')
    colors = ['r' if l else 'b' for l in labels]
    ax.scatter(features[:, 0], features[:, 1], marker='o', c=colors, s=100, alpha = 0.5)
    if W is not None:
        min_x = min(features[:,0])
        max_x = max(features[:,1])
        min_y = min(features[:,1])*(1-.1)
        max_y = max(features[:,1])*(1+.1)
        cx = np.array([min_x,max_x],dtype=np.float32)
        cy = (0.5-W[0]*cx-b)/W[1]
        ax.plot(cx,cy,'g')
        ax.set_ylim(min_y,max_y)
    fig.show()

In [ ]:
%matplotlib inline
plot_dataset(train_x, train_labels)

## Verileri Normalleştirme

Eğitimden önce, girdi özniteliklerimizi [0,1] (veya [-1,1]) standart aralığına getirmek yaygındır. Bunun tam nedenlerini kursun ilerleyen kısımlarında tartışacağız, ancak kısaca nedeni şudur. Ağımız üzerinden akan değerlerin çok büyük veya çok küçük olmasını önlemek istiyoruz ve normalde tüm değerleri 0'a yakın küçük bir aralıktaki tutmada hemfikiriz. Böylece ağırlıkları küçük rastgele sayılarla ilkliyoruz ve sinyalleri aynı değer aralığında tutuyoruz.

Verileri normalleştirirken en küçük değeri çıkarmamız ve aralığa bölmemiz gerekiyor. Eğitim verilerini kullanarak en küçük değeri ve değer aralığını hesaplıyoruz ve ardından eğitim kümesindeki aynı minimum/aralık değerlerini kullanarak test/geçerleme veri kümesini normalleştiriyoruz. Bunun nedeni, gerçek hayatta sadece eğitim kümesini bileceğiz ve ağın tahmin etmesi istenecek gelen tüm yeni değerleri değil. Bazen yeni değer [0,1] aralığının dışına çıkabilir, ancak bu çok önemli değildir.

In [ ]:
train_x_norm = (train_x-np.min(train_x,axis=0)) / (np.max(train_x,axis=0)-np.min(train_x,axis=0))
test_x_norm = (test_x-np.min(train_x,axis=0)) / (np.max(train_x,axis=0)-np.min(train_x,axis=0))

## Tek Katmanlı Ağ Eğitimi (Algılayıcı)

Çoğu durumda, bir sinir ağı bir dizi katman olacaktır. Keras'ta `Sequential` (dizili) model kullanılarak aşağıdaki şekilde tanımlanabilir:

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(2,)))
model.add(keras.layers.Dense(1))
model.add(keras.layers.Activation(keras.activations.sigmoid))

model.summary()

Burada önce modeli oluşturuyoruz ve ardından ona katmanlar ekliyoruz:
* İlk `Input` (Girdi) katmanı (aslında katman diyemeyiz) ağın girdi boyutunun beyanını içerir.
* `Dense` (yoğun) katman, eğitilebilir ağırlıkları içeren gerçek algılayıcıdır.
* Son olarak, ağın sonucunu 0-1 aralığına getirmek (onu olasılık yapmak) için *sigmoid* `Activation` (etkinleştirme) işlevine sahip bir katman vardır.

Girdi boyutu ve etkinleştirme işlevi, kısa olması için doğrudan `Dense` katmanında da belirtilebilir:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(1,input_shape=(2,),activation='sigmoid'))
model.summary()

Modeli eğitmeden önce, onu **derlememiz** gerekir, bu da esasen şunu belirtmek anlamına gelir:
* **Kayıp fonksiyonu**, kaybın nasıl hesaplandığını tanımlar. İki sınıflı sınıflandırma problemimiz olduğu için *ikili çapraz entropi kaybı* kullanacağız.
* **Optimizer (eniyileyici)** kullanmak için. En basit seçenek, *rasgele gradyan inişi* için `sgd`'yi kullanmaktır veya `adam` gibi daha karmaşık eniyileyicileri kullanabilirsiniz.
* Eğitimimizin başarısını ölçmek için kullanmak istediğimiz **metrikler**. Sınıflandırma görevi olduğundan, iyi bir metrik `Accuracy` (doğruluk) (veya kısaca `acc`) olacaktır.

Kaybı, metrikleri ve eniyileyiciyi dizgiler (string) olarak veya Keras çerçevesinden bazı nesneler sağlayarak belirtebiliriz. Örneğimizde, modelimizin öğrenme oranına ince ayar yapmak için `learning_rate` parametresini belirtmemiz gerekiyor ve bu nedenle Keras SGD eniyileyicisinin tam adını sağlıyoruz.

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.2),loss='binary_crossentropy',metrics=['acc'])

Modeli derledikten sonra `fit` metodunu çağırarak asıl eğitimi yapabiliriz. En önemli parametreler şunlardır:
* `x` ve `y` sırasıyla eğitim verilerini, öznitelikleri ve etiketleri belirtir.
* Her dönemde geçerlemenin yapılmasını istiyorsak, bir dizi özellik ve etiket olacak olan `validation_data` parametresini belirtebiliriz.
* `epochs`, dönemlerin sayısını belirtti.
* Eğitimin minigruplarda gerçekleşmesini istiyorsak, `batch_size` parametresini belirtebiliriz. Ayrıca verileri `x`/`y`/`validation_data`'ya aktarmadan önce manuel olarak önceden toplu işleyebilirsiniz; bu durumda `batch_size` gerekmez.

In [ ]:
model.fit(x=train_x_norm,y=train_labels,validation_data=(test_x_norm,test_labels),epochs=10,batch_size=1)

Eğitimi nasıl etkilediklerini görmek için farklı eğitim parametreleriyle denemeler yapabilirsiniz:
* `batch_size` ayarının çok büyük olması (veya hiç belirtilmemesi) daha az kararlı eğitime neden olabilir, çünkü düşük boyutlu verilerle küçük toplu iş boyutları her bir özel durum için gradyanın daha kesin yönünü sağlar.
* Çok yüksek `learning_rate` (öğrenme oranı), aşırı öğrenme ile veya daha az kararlı sonuçlarla sonuçlanabilirken, çok düşük öğrenme oranı, sonuca ulaşmanın daha fazla dönem alacağı anlamına gelir.

> Ağı daha fazla eğitmek için `fit` (oturt) işlevini arka arkaya birkaç kez çağırabileceğinizi unutmayın. Eğitime sıfırdan başlamak istiyorsanız - hücreyi model tanımıyla yeniden çalıştırmanız gerekir.

Eğitimimizin işe yaradığından emin olmak için iki sınıfı ayıran çizgiyi çizelim. Ayırma çizgisi $W\times x + b = 0.5$ denklemiyle tanımlanır.

In [ ]:
plot_dataset(train_x,train_labels,model.layers[0].weights[0],model.layers[0].weights[1])

## Eğitim grafiklerini çizme

`fit` işlevi, sonuç olarak, her dönemdeki kaybı ve metrikleri gözlemlemek için kullanılabilen `history` (tarih) nesnesini döndürür. Aşağıdaki örnekte küçük bir öğrenme oranı ile eğitime yeniden başlayacağız ve kayıp ve doğruluğun nasıl davrandığını gözlemleyeceğiz.

> `Sequential` modeli tanımlamak için biraz farklı sözdizimi kullandığımızı **unutmayın**. Katmanları tek tek eklemek (`add`) yerine, ilk etapta modeli oluştururken katmanların listesini de belirtebiliriz - bu biraz daha kısa sözdizimidir ve onu kullanmayı tercih edebilirsiniz.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid')])
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.05),loss='binary_crossentropy',metrics=['acc'])
hist = model.fit(x=train_x_norm,y=train_labels,validation_data=(test_x_norm,test_labels),epochs=10,batch_size=1)

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])

## Çok Sınıflı Sınıflandırma

Bir çok sınıflı sınıflandırma problemini çözmeniz gerekiyorsa, ağınızın birden fazla çıktısı olacaktır - ki $C$ sınıflarının sayısına karşılık gelir. Her çıktı belirli bir sınıfın olasılığını içerecektir.

> Aynı şekilde ikili sınıflandırma gerçekleştirmek için iki çıktılı bir ağ da kullanabileceğinizi unutmayın. Şimdi tam olarak bunu göstereceğiz.

Bir ağdan bir $p_1,\dots, p_C$ olasılıkları kümesi çıktılamasını beklediğinizde, hepsinin toplamının 1'e eşit olmasına ihtiyacımız var. Bunu sağlamak için, son katmanda son etkinleştirme fonksiyonu olarak `softmax` kullanıyoruz. **Softmaks** bir vektör girdisi alır ve bu vektörün tüm bileşenlerinin olasılıklara dönüştürülmesini sağlar.

Ayrıca, ağın çıktısı $C$ boyutlu bir vektör olduğundan, aynı forma sahip etiketlere ihtiyacımız var. Bu, $i$ sınıfının değeri, $i$. konumda 1 ile sıfırlardan oluşan bir vektöre dönüştürüldüğünde **bire bir kodlama** kullanılarak gerçekleştirilebilir.

Sinir ağının olasılık çıktısını beklenen bire bir kodlanmış etiketle karşılaştırmak için **çapraz entropi kaybı** işlevini kullanırız. İki olasılık dağılımı alır ve ne kadar farklı olduklarının bir değerini verir.

O halde $C$ sınıfı olan çok sınıflı sınıflandırma için yapmamız gerekenleri özetlemek gerekirse:
* Ağın son katmanında $C$ adet nöronları olmalıdır.
* Son etkinleştirme işlevi **softmaks** olmalıdır.
* Kayıp, **çapraz entropi kaybı** olmalıdır.
* Etiketler **bire bir kodlamaya** dönüştürülmelidir (bu, `numpy` kullanılarak veya Keras utils `to_categorical` kullanılarak yapılabilir).

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(5,input_shape=(2,),activation='relu'),
    keras.layers.Dense(2,activation='softmax')
])
model.compile(keras.optimizers.Adam(0.01),'categorical_crossentropy',['acc'])

# Bire bir kodlamaya dönüştürmenin iki yolu
train_labels_onehot = keras.utils.to_categorical(train_labels)
test_labels_onehot = np.eye(2)[test_labels]

hist = model.fit(x=train_x_norm,y=train_labels_onehot,
                 validation_data=[test_x_norm,test_labels_onehot],batch_size=1,epochs=10)

### Seyrek Kategorik Çapraz Entropi

Çok sınıflı sınıflandırmadaki etiketler genellikle sınıf numaralarıyla temsil edilir. Keras ayrıca, sınıf numarasının bire bir vektörler değil, tamsayılar olmasını bekleyen **seyrek kategorik çapraz entropi** adı verilen başka bir tür kayıp işlevini de destekler. Bu tür bir kayıp fonksiyonunu kullanarak eğitim kodumuzu basitleştirebiliriz:

In [ ]:
model.compile(keras.optimizers.Adam(0.01),'sparse_categorical_crossentropy',['acc'])
model.fit(x=train_x_norm,y=train_labels,validation_data=[test_x_norm,test_labels],batch_size=1,epochs=10)

## Çok Etiketli Sınıflandırma

> **Note** that this is very similar to using **different neural networks** to do binary classification for each particular class - only the initial part of the network (up to final classification layer) is shared for all classes.

Bazen nesnelerimizin aynı anda iki sınıfa ait olabileceği durumlar olur. Örnek olarak, resimdeki kediler ve köpekler için bir sınıflandırıcı geliştirmek istediğimizi, ancak hem kedilerin hem de köpeklerin bulunduğu durumlara da izin vermek istediğimizi varsayalım.

Çok etiketli sınıflandırma ile, bire bir kodlanmış vektör yerine, girdi örneğiyle ilgili tüm sınıflara karşılık gelen 1 konumunda bir vektöre sahip olacağız. Bu nedenle, ağın çıktısı tüm sınıflar için normalleştirilmiş olasılıklara sahip olmamalı, bunun yerine her sınıf için ayrı ayrı olmalıdır - bu, **sigmoid** etkinleştirme fonksiyonunun kullanılmasına karşılık gelir. Çapraz entropi kaybı hala bir kayıp fonksiyonu olarak kullanılabilir.

> Bunun her bir sınıf için ikili sınıflandırma yapmak için **farklı sinir ağları** kullanmaya çok benzer olduğuna **dikkat edin** - tüm sınıflar için ağın yalnızca ilk kısmı (son sınıflandırma katmanına kadar) paylaşılır.

## Sınıflandırma Kaybı Fonksiyonlarının Özeti

Ağın son katmanındaki kayıp fonksiyonu ve etkinleştirme fonksiyonuna göre ikili, çok sınıflı ve çok etiketli sınıflandırmanın farklılık gösterdiğini gördük. Yeni öğrenmeye başlıyorsanız, biraz kafa karıştırıcı olabilir, ancak burada aklınızda bulundurmanız gereken birkaç kural vardır:
* Eğer ağda bir tane çıktılı (**ikili sınıflandırma**) varsa **sigmoid** etkinleştirme işlevini kullanırız, **çok sınıflı sınıflandırma** içinse  **softmaks**.
* Çıktı sınıfı bire bir kodlama olarak temsil edilirse, kayıp işlevi **çapraz entropi kaybı** (kategorik çapraz entropi) eğer çıktı sınıf numarası içeriyorsa **seyrek kategorik çapraz entropi**  olacaktır. **İkili sınıflandırma** için **ikili çapraz entropi** kullanın (**logaritmik kayıp** ile aynıdır).
* **Çok etiketli sınıflandırma**, aynı anda birkaç sınıfa ait bir nesneye sahip olabileceğimiz zamandır. Bu durumda, etiketleri bire bir kodlama kullanarak kodlamamız ve etkinleştirme fonksiyonu olarak **sigmoid** kullanmamız gerekir, böylece her sınıf olasılığı 0 ile 1 arasında olur.

| Sınıflandırma | Etiket Formatı | Etkinleştirme Fonksiyonu | Kayıp |
|---------------|-----------------------|-----------------|----------|
| İkili         | 1. sınıf olasılığı | sigmoid | ikili çapraz entropi |
| İkili         | Bire bir kodlama (2 çıktılı) | softmax | kategorik çapraz entropi |
| Çok sınıflı   | Bire bir kodlama | softmaks | kategorik çapraz entropi |
| Çok sınıflı   | Sınıf sayısı | softmaks | seyrek kategorik çapraz entropi |
| Çok sınıflı   | Bire bir kodlama | sigmoid | kategorik çapraz entropi |


**Görev**:
MNIST el yazısı rakamları için bir sınıflandırıcı eğitmede Keras'ı kullanın:
* Keras'ın MNIST dahil bazı standart veri kümeleri içerdiğine dikkat edin. MNIST'i Keras'tan kullanmak için yalnızca birkaç satır koda ihtiyacınız vardır (daha fazla bilgi [burada](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist)dır).
* Farklı sayıda katman/nöron, etkinleştirme işlevleriyle birkaç ağ yapılandırması deneyin.

Ulaşabildiğiniz en iyi doğruluk nedir?

## Ana Fikirler

* **Keras**, yeni başlayanlar için gerçekten tavsiye edilir, çünkü katmanlardan ağları oldukça kolay bir şekilde oluşturmaya ve ardından sadece birkaç satır kodla eğitmeye olanak tanır.
* Standart olmayan bir mimari gerekiyorsa, Tensorflow'u biraz daha derinlemesine öğrenmeniz gerekir. Veya birinden özel mantığı bir Keras katmanı olarak uygulamasını isteyebilir ve ardından bunu Keras modellerinde kullanabilirsiniz.
* PyTorch'a da bakmak ve yaklaşımları karşılaştırmak iyi bir fikirdir.

Keras üzerine Keras ve Tensorflow 2.0'nin yaratıcısından güzel bir örnek not defterini [burada](https://t.co/k694J95PI8) bulabilirsiniz.